##    ĐỒ ÁN CUỐI KỲ MÔN: NHẬP MÔN KHOA HỌC DỮ LIỆU (HCMUS)
<ul><h2>GV: TRẦN TRUNG KIÊN <br> SV: Nguyễn Ngọc Bình - Nguyễn Duy Khải</h2></ul>
 

<p></p>

### ĐỒ ÁN NÀY THỰC HIỆN MỘT CÂU HỎI: Đánh giá chỉ số IMDB của những bộ phim gần đây (2019-2020)

### CÂU HỎI NÀY CÓ Ý NGHĨA GÌ?
### <ul> Sẽ mang chọn được phim hay mà xem <br> <br>Nhà làm phim sẽ biết những thứ gì họ cần để thu hút người xem ngoài kich bản hay</ul>


### QUY TRÌNH THỰC HIỆN
<h3> <ul>
        Crawl dữ liệu từ web <br><br>
        Tiền xử lý dữ liệu <br><br>
        Chọn mô hình cho bài toán <br><br>
        Hạn chế và khó khăn   <br><br>
        Tài liệu kham khảo 
</ul></h3>


### CÁC THƯ VIỆN SỬ DỤNG

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import datetime as dt 
import time
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

### I. CRAWl DỮ LIỆU TỪ WEBSITE

<ul> Dữ liệu đánh giá phim nhóm lấy từ website  <a href="https://www.imdb.com"> <b>IMDB</b></a>, đây là một website đánh giá và review phim khá uy tín mà nhóm đã tìm hiểu được. Trong hầu hết website phim thì luôn đề cập điểm imdb (là cái điểm cạnh ngôi sao mà những bộ phim hay để), để bước tiếp thì nhóm sẽ nói sơ về điểm imdb <b>(Internet Movie Database) </b>  là một thư viện điện ảnh cung cấp nhiều chủ đề như phim, đạo diễn, diễn viên, phim và nhiều loại hình giải trí. Nổi bậc hơn là nơi đánh giá các bộ phim trên thang điểm 10 với các tiêu chí: kịch bản, diễn xuất, bối cảnh, hiệu ứng kỹ xảo và kỹ thuât quy phim. Điểm này được đánh giá bởi đông đảo những người xem nên độ tin cậy cũng khá ổn.</ul>

<ul>Một bộ phim gồm nhiều thông tin, nhưng do mục tiêu nên nhóm chỉ lấy tên phim, xếp hạng của phim, thể loại, đạo diễn và các diễn viên có tiếng. Nhận thấy có 2 cái xếp hạng, 1 là IMDB với 1 cái là metascore, nhóm chọn IMDB </ul>

![Image of Yaktocat](https://scontent.fhan4-1.fna.fbcdn.net/v/t1.15752-9/135584979_1159547281146246_6991992338298078328_n.jpg?_nc_cat=108&ccb=2&_nc_sid=ae9488&_nc_ohc=M0cf_xn1D_sAX8B4IzV&_nc_ht=scontent.fhan4-1.fna&oh=2f6e05a73d2232473e5ed90afa97cef4&oe=6020D28A)

<ul>
    <h3>Lấy dữ liệu bộ phim trong hai năm gần đây </h3><br>
    <ul>
        Trong phạm vi giới hạn của nhóm, nhóm chỉ lấy vài thông tin như là tên movie, thời gian chiếu, thể loại, tên đạo diễn và ngôi sao tham dự và hạng
    </ul>
</ul>



<h3><ul>Năm 2019</ul></h3>

<h3><ul>Năm 2020</ul></h3>

In [21]:
def add_dict_value_other(s,dict_temp): 
    s=s[s.find('Director'):].replace('\n','').replace('Votes','|Votes').split('|')
    for item in s:
        temp =item.strip().split(':')
        if(temp[0]=='Director' or temp[0]=='Stars'):
            dict_temp[temp[0]]=temp[1] 

In [33]:
num_title= 1
max_num =9881
check_break=0
action_movie=[]
num_movies=0
while(1):
    if(num_title==1):
        link_page=f'https://www.imdb.com/search/title/?title_type=feature&year=2020-01-01,2020-12-31&sort=year,desc'
    else:
        link_page = f'https://www.imdb.com/search/title/?title_type=feature&year=2020-01-01,2020-12-31&sort=year,desc&start={num_title}&ref_=adv_nxt'
    #lấy nội dung lưu vào dict
    html_text = requests.get(link_page).text
    tree = BeautifulSoup(html_text, 'html.parser')
    movies = tree.find_all('div',{'class':'lister-item mode-advanced'})
    for movie in movies:
        num_movies+=1
        dict_temp_movie ={}
        header=movie.find('h3',{'class':'lister-item-header'})
        dict_temp_movie['Name']= header.a.text
        text_muted =movie.find('p',{'class':'text-muted '})
        try:
            runtime=int(text_muted.find('span','runtime').text.split()[0])
        except:
            runtime=0
        dict_temp_movie['Runtime']=runtime
        try:
            genre=text_muted.find('span','genre').text.strip().strip('\n').strip(' ')#.split(', ')
        except:
            genre=None
        dict_temp_movie['Genre']=genre
        ratings_bar =movie.find('div',{'class':'ratings-bar'})
        try:
            IMDB_score =float(ratings_bar.find('div',{'class':'inline-block ratings-imdb-rating'})['data-value'])
            dict_temp_movie['IMDB'] = IMDB_score
            sort_num_votes_visible = movie.find('p','sort-num_votes-visible').text
            add_dict_value_other(movie.text,dict_temp_movie)
            action_movie.append(dict_temp_movie)
        except:
            pass
    time.sleep(1)
    if(num_movies>=max_num):
        break
    num_title+=50
df = pd.DataFrame(action_movie)
df.to_csv('movies2020.csv',index=False)

### II. TIỀN XỬ LÝ DỮ LIỆU
<ul>Đọc hai file csv lên để gộp và xử lý</ul>

### <ul> <l>  Chia tập dữ liệu </l> </ul>

In [42]:
df2020= pd.read_csv('movies2020.csv')
df2019=pd.read_csv('movie2019.csv')
df2020.rename(columns = {'Name':'Title','IMDB':'Rating','Director':'Directors'}, inplace = True)
df=pd.concat([df2019,df2020],ignore_index=True)

<ul><ul>Xử lý dữ liệu trống bằng thay thế các từ cần thiết, xóa những dòng dữ liệu bất thường, tạo dữ liệu để thực hiện bài toán, xóa luôn cột Title (tên phim) vi nó không mang giá trị, ý nghĩa</ul></ul>

<h3> <ul>   Đánh giá phim </ul></h3>

<ul><ul>
    Với tư cách là người xem phim, tìm những phim đáng để xem thì chúng ta sẽ không quan tâm quá nhiều đến việc điểm <b>IMDB</b> như thế nào, mà chỉ quan tâm xem từ đó mình có thể chọn phim hay được không?<br>
    Do đó, qua quá trình tìm hiểu nhiều nguồn dữ liệu trên <b><a href='https://www.google.com/'>Google</a></b> thì có vài nhận định về khoảng <b>IMBD </b> như sau:
    <ol>Phim quá xuất sắc: IMBD >=8.0</ol>
    <ol>Phim quá hay:  IMBD trong khoảng 7.0 - 8.0</ol>
    <ol>Phim  bình thường: IMBD trong khoảng 5.0 - 7.0</ol>
    <ol>Phim tệ: IMBD trong khoảng 3.0 - 5.0</ol>
    <ol> Phim kinh khủng: IMBD dưới 3.0 </ol>
</ul></ul>

<h3><ul>Chuẩn hóa lại cột Rating</ul></h3>

In [39]:
def rating_format(rating):
    new_rating=""
    if(rating>=8.0):
        new_rating=1
    elif(rating>=7.0):
        new_rating=2
    elif(rating>=5):
        new_rating=3
    elif(rating>=3):
        new_rating=4
    else:
        new_rating=5
    return new_rating  


In [43]:
df.dropna(subset=['Runtime'],inplace=True)
df= df.reset_index(drop=True)
df.loc[:,'Genre']=df.loc[:,'Genre'].replace(np.nan,'other')
df.drop(index=df[df.Runtime >= 1000].index,inplace= True)
X_df = df.drop(["Rating","Title"], axis=1)
y_sr = df["Rating"]
y_sr=y_sr.apply(rating_format)

In [44]:
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)

### <ul>Tạo pipeline tiền xử lý dữ liệu </ul>

<ul><ul><strong>Lấy dữ liệu top 100 stars trên IMDB và top 250 director trên IMDB và chuyển về dạng có hạng</strong></ul></ul>

In [6]:
stars_top100=['Karen Gillan', 'Bryce Dallas Howard', 'Alicia Vikander', 'Tom Hardy', 'Rebecca Ferguson', 'Lily James', 'Jessica Barden', 'Hannah John-Kamen', 'Josh Brolin', 'Tessa Thompson', 'Alexandra Daddario', 'Brie Larson', 'Carla Gugino', 'Emilia Clarke', 'Evangeline Lilly', 'Margot Robbie', 'Martha Higareda', 'Michael B. Jordan', 'Ruby Rose', 'Olivia Cooke', 'Morena Baccarin', 'Yvonne Strahovski', 'Julia Garner', 'Noah Centineo', 'Ana de Armas', 'Kiernan Shipka', 'John Krasinski', 'Meghan Markle', 'Jacob Elordi', 'Parker Posey', 'Rami Malek', 'Vanessa Kirby', 'Bonnie Aarons', 'Sonoya Mizuno', 'Zendaya', 'Debby Ryan', 'Gal Gadot', 'Emily Blunt', 'Timothée Chalamet', 'Naomi Scott', 'Jennifer Lawrence', 'Joey King', 'Tom Cruise', 'Joel Kinnaman', 'Jessica Chastain', 'Alden Ehrenreich', 'Bill Skarsgård', 'Zazie Beetz', 'Frances McDormand', 'Kate Siegel', 'Burt Reynolds', 'Amber Heard', 'Alison Brie', 'Chadwick Boseman', 'Daisy Ridley', 'Steve Wilder', 'Eliza Dushku', 'Lady Gaga', 'Zoey Deutch', 'Sally Hawkins', 'Jackson Odell', 'Allison Mack', 'Pom Klementieff', 'Jimmy Bennett', 'Abbie Cornish', 'Saoirse Ronan', 'Letitia Wright', 'Elizabeth Olsen', 'Dwayne Johnson', 'Gemma Chan', 'Rachel Brosnahan', 'Claire Foy', 'Milly Shapiro', 'Michelle Williams', 'Henry Cavill', 'Amy Adams', 'Chris Hemsworth', 'Lana Condor', 'Jason Momoa', 'Richard Madden', 'Joel Edgerton', 'Cate Blanchett', 'Alice Eve', 'Gary Oldman', 'Chris Pratt', 'Tom Holland', 'Hailee Steinfeld', 'Mina Sundwall', 'Ryan Reynolds', 'Cristin Milioti', 'Olivia Munn', 'Lucy Davis', 'Blake Lively', 'Mackenzie Foy', 'Amanda Fuller', 'Scarlett Johansson', 'Dichen Lachman', 'Sam Rockwell', 'Sofia Boutella', 'Evan Rachel Wood']
directors_top250= ['Christopher Nolan', 'Peter Jackson', 'Quentin Tarantino', 'Martin Scorsese', 'James Cameron', 'Steven Spielberg', 'David Fincher', 'Lee Unkrich', 'Ridley Scott', 'George Lucas', 'Stanley Kubrick', 'Zack Snyder', 'Ethan Coen', 'Joel Coen', 'Gore Verbinski', 'Francis Ford Coppola', 'Bryan Singer', 'Pete Docter', 'Matthew Vaughn', 'Anthony Russo', 'Joe Russo', 'Guy Ritchie', 'Robert Zemeckis', 'Sam Raimi', 'J.J. Abrams', 'Brad Bird', 'Alfonso Cuarón', 'Lana Wachowski', 'Lilly Wachowski', 'Doug Liman', 'David Yates', 'Luc Besson', 'Clint Eastwood', 'Chris Columbus', 'Paul Greengrass', 'Marc Webb', 'Tim Burton', 'Francis Lawrence', 'Ron Howard', 'Michael Bay', 'Sam Mendes', 'Danny Boyle', 'John McTiernan', 'Neil Burger', 'Denis Villeneuve', 'Andrew Adamson', 'James Wan', 'Edgar Wright', 'Roland Emmerich', 'John Lasseter', 'David O. Russell', 'Alfred Hitchcock', 'M. Night Shyamalan', 'Todd Phillips', 'Barry Sonnenfeld', 'Phil Lord', 'Christopher Miller', 'Steven Soderbergh', 'Antoine Fuqua', 'Guillermo del Toro', 'Darren Aronofsky', 'Hayao Miyazaki', 'Alejandro G. Iñárritu', 'Mel Gibson', 'Joss Whedon', 'Louis Leterrier', 'James Mangold', 'Sergio Leone', 'Joe Johnston', 'Michael Mann', 'Frank Darabont', 'Tony Scott', 'Andrew Niccol', 'Mike Newell', 'Brian De Palma', 'Tom Shadyac', 'Andrew Stanton', 'Wes Anderson', 'Adam McKay', 'Terry Gilliam', 'Ang Lee', 'Justin Lin', 'Jon Favreau', 'Richard Linklater', 'Ben Affleck', 'Ben Stiller', 'Len Wiseman', 'Stephen Sommers', 'Woody Allen', 'Peter Berg', 'Martin Campbell', 'Duncan Jones', 'Martin McDonagh', 'Cameron Crowe', 'Paul Thomas Anderson', 'Brett Ratner', 'Joel Schumacher', 'Rob Reiner', 'Paul Verhoeven', 'Neill Blomkamp', 'David Ayer', 'Bobby Farrelly', 'Peter Farrelly', 'Jason Reitman', 'Marc Forster', 'Ron Clements', 'John Musker', 'Bill Condon', 'Jay Roach', 'Shane Black', 'Carlos Saldanha', 'Baz Luhrmann', 'Peter Weir', 'Peter Segal', 'Matt Reeves', 'Robert Rodriguez', 'Pierre Coffin', 'Sylvester Stallone', 'Simon West', 'Dennis Dugan', 'Paul Feig', 'Nancy Meyers', 'Shawn Levy', 'Tom Tykwer', 'Joe Wright', 'Alex Proyas', 'Tom McGrath', 'Jaume Collet-Serra', 'Byron Howard', 'Chris Williams', 'F. Gary Gray', 'Paul W.S. Anderson', 'Jonathan Liebesman', 'Lasse Hallström', 'D.J. Caruso', 'Chris Weitz', 'Wolfgang Petersen', 'Roman Polanski', 'Edward Zwick', 'Kevin Smith', 'Ivan Reitman', 'Spike Jonze', 'McG', 'Nicholas Stoller', 'David Lynch', 'Robert Luketic', 'Seth MacFarlane', 'Scott Derrickson', 'George Miller', 'Oliver Stone', 'David Dobkin', 'Chris Renaud', 'Richard Donner', 'Greg Mottola', 'Martin Brest', 'David Slade', 'Olivier Megaton', 'Dean DeBlois', 'Chris Sanders', 'David Twohy', 'Jon Turteltaub', 'Wes Craven', 'Eric Darnell', 'Conrad Vernon', 'David Cronenberg', 'Charles Chaplin', 'Tom Hooper', 'Billy Wilder', 'Rob Cohen', 'David Leitch', 'J.A. Bayona', 'Kenneth Branagh', 'Kathryn Bigelow', 'Larry Charles', 'Joe Carnahan', 'Anthony Minghella', 'Rob Marshall', 'Robert Schwentke', 'Darren Lynn Bousman', 'Rawson Marshall Thurber', 'Sidney Lumet', 'David Zucker', 'Mark Neveldine', 'Brian Taylor', 'Christopher McQuarrie', 'Brian Helgeland', 'Dean Parisot', 'Frank Coraci', 'Kirk Wise', 'John Landis', 'Jan de Bont', 'Judd Apatow', 'Clyde Geronimi', 'Kevin Reynolds', 'Alexander Payne', 'Keenen Ivory Wayans', 'Bennett Miller', 'Milos Forman', 'Peyton Reed', 'Barry Levinson', 'Nick Cassavetes', 'Mel Brooks', 'Wilfred Jackson', 'Hamilton Luske', 'Seth Gordon', 'Stephen Daldry', 'Baltasar Kormákur', 'John G. Avildsen', 'John Singleton', 'John Carpenter', 'Renny Harlin', 'John Hughes', 'Garry Marshall', 'Taylor Hackford', 'Lars von Trier', 'Phillip Noyce', 'Brad Silberling', 'Anne Fletcher', 'Jack Kinney', 'Bill Roberts', 'Ben Sharpsteen', 'Jerry Zucker', 'Yimou Zhang', 'Terry Jones', 'Richard Curtis', 'Wolfgang Reitherman', 'Mike Nichols', 'Nicolas Winding Refn', 'Jake Kasdan', 'John Woo', 'Paul Weitz', 'Rob Minkoff', 'Akira Kurosawa', 'Jonathan Demme', 'Chuck Russell', 'Chris Wedge', 'John Moore', 'Harold Ramis', 'Rian Johnson', 'Andy Tennant', 'Craig Gillespie', 'Terence Young', 'Gregory Hoblit', 'Nora Ephron', 'Jean-Jacques Annaud', 'Mark Waters', 'Tim Story', 'Sofia Coppola', 'James Gunn', 'Morten Tyldum']
def create_rank_dict(rank_list):
    rank_dict = {}
    for i in range(len(rank_list)):
        rank_dict[rank_list[i]]=i+1
    return rank_dict
rank_directors= create_rank_dict(directors_top250)
rank_stars = create_rank_dict(stars_top100)

<ul>
    <ul>
        <strong>Tạo transformer tiền xử lý các cột dữ liệu</strong>
    </ul>
</ul>

<ul><ul><ul><u><b>Xử lý cột Genre:</b></u> Tách dữ liệu cột Genre thành các cột mới, với các cột mới là cacs cột phổ biến được gợi ý tren website <a href="https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm"> <b>IMDB</b></a>, với bộ phim nào mà có thuộc tính thể loại đó sẽ đánh là 1 và ngược lại là 0
    
</ul></ul></ul>

In [7]:
genres = ['Action','Adventure','Animation','Biography','Comedy','Crime','Drama','Family','Fantasy','Film-Noir','History',
          'Horror','Music','Musical','Mystery','Romance','Sci-Fi','Sport','Thriller','War','Western']
def GenreReplacement(string):
    #print(string)
    ls = string.split(', ')
    new_ls = [x if x in genres else 'other' for x in ls]
    return new_ls
class GenreTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        result = X_df.copy()
        result['Genre'] = result['Genre'].apply(GenreReplacement)
        result[genres] = 0
        for genre in genres:
            result.loc[:,[genre]] = result['Genre'].apply(lambda x: 1 if genre in x else 0)
        result.drop(columns=['Genre'], inplace = True)
        return result

<ul><ul><ul><u><b>Xử lý cột Directors:</u></b> Về cơ bản thì xem phim, đạo diễn nào nổi tiếng thì phim thường sẽ hay, nên cột này sẽ so sánh xem thứ hạng Director, Director nào mà quá hạng thì sẽ coi như là hạng 251 
    
</ul></ul></ul>

In [8]:
def Directors_Replacement(director):
    if(director in directors_top250 ):
        score =rank_directors[director]
    else:
        score= 251
    return score       
class Directors_Transformer(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        result = X_df.copy()
        result['Directors'] = result['Directors'].apply(Directors_Replacement)
        return result

<ul><ul><ul><u><b>Xử lý cột Directors:</u></b> Về cơ bản thì xem phim, đạo diễn nào nổi tiếng thì phim thường sẽ hay, nên cột này sẽ so sánh xem thứ hạng Director, Director nào mà quá hạng thì sẽ coi như là hạng 251 
    
</ul></ul></ul>

In [9]:
def p_Stars_Replacement(stars):
    stars_list = stars.split(',')
    n=0
    for star in stars_list:
        if star in stars_top100:
            n+=1
    p=(n+1)/(len(stars_list)+1)
    return  p
def score_Stars_Replacement(stars):
    stars_list = stars.split(',')
    score= 102*4
    for star in stars_list:
        if star in stars_top100:
            score=score-rank_stars[star]
    return score
               
class Stars_Transformer(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_titles=1):
        self.num_top_titles = num_top_titles
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        result = X_df.copy()
        result.loc[:,'Stars']=result.loc[:,'Stars'].replace(np.nan,'other')
        result['p_stars']=result['Stars'].apply(p_Stars_Replacement)
        result['Stars'] = result['Stars'].apply(score_Stars_Replacement)
        return result

<ul><ul><ul><u><b>Xử lý cột Runtime:</u></b> Một bộ phim thường có nhiều thơi gian, có phim ngắn, phim dài và phim trung bình, do không có một cái gì để chuẩn mực đo lường nên nhóm sẽ chia thời gian thành nhiều <b>bin</b> khác nhau và thực hiện bởi thuật toán <b><a href= "https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html"> KBinsDiscretizer </a></b> để chia thành k giỏ
    
</ul></ul></ul>

In [10]:
class Runtime_Transformer(BaseEstimator, TransformerMixin):
    def __init__(self, k_bin=3):
        self.k_bin= k_bin
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        result = X_df.copy()
        enc = KBinsDiscretizer(n_bins=self.k_bin,encode='ordinal', strategy='kmeans')
        result['Runtime']=enc.fit_transform(X_df[['Runtime']])
        return result

<ul>
    <ul>
        <strong>Tạo pipeline tiền xử lý dữ liệu với các Transformer ở trên và dùng StandardScaler</strong>
    </ul>
</ul>

In [11]:
preprocess_pipeline = Pipeline(
    [
        ('GenreTransformer',GenreTransformer()),
        ('Directors_Transformer', Directors_Transformer()),
        ('Stars_Transformer', Stars_Transformer()),
        ('Runtime_Transformer',Runtime_Transformer()),
        ('StandardScaler',StandardScaler())
    ]
)
preprocess_pipeline

Pipeline(steps=[('GenreTransformer', GenreTransformer()),
                ('Directors_Transformer', Directors_Transformer()),
                ('Stars_Transformer', Stars_Transformer()),
                ('Runtime_Transformer', Runtime_Transformer()),
                ('StandardScaler', StandardScaler())])

<ul>
    <ul>
        <strong>Sử dụng pipeline vừa tạo để áp dụng lên tập train và validation</strong>
    </ul>
</ul>

In [12]:
preprocessed_train_X=preprocess_pipeline.fit_transform(train_X_df)
preprocessed_val_X=preprocess_pipeline.transform(val_X_df)

### III. CHỌN MÔ HÌNH CHO BÀI TOÁN

<ul><ul>Tiến hành xây dụng một pipeline để áp dụng model <b>MLPClassifier</b> cho bài toán </ul></ul>

In [31]:
from sklearn import linear_model
from sklearn.linear_model import ElasticNet,LinearRegression
from sklearn.preprocessing import PolynomialFeatures
model =MLPClassifier(hidden_layer_sizes=(20,), activation='logistic',
                                 solver='lbfgs', max_iter=2500, random_state=0)
full_pipeline = make_pipeline(preprocess_pipeline,model)
full_pipeline

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('GenreTransformer', GenreTransformer()),
                                 ('Directors_Transformer',
                                  Directors_Transformer()),
                                 ('Stars_Transformer', Stars_Transformer()),
                                 ('Runtime_Transformer',
                                  Runtime_Transformer(k_bin=2)),
                                 ('StandardScaler', StandardScaler())])),
                ('mlpclassifier',
                 MLPClassifier(activation='logistic', hidden_layer_sizes=(20,),
                               max_iter=2500, random_state=0,
                               solver='lbfgs'))])

In [32]:
train_errs = []
val_errs = []
alphas = [0.1, 1, 10, 100]
kbins = [2,3, 4, 5, 6]
best_val_err = float('inf'); best_alpha = None; best_k_bin = None
for alpha in alphas:
    for bins in kbins:
        # YOUR CODE HERE
        full_pipeline.set_params(pipeline__Runtime_Transformer__k_bin=bins, mlpclassifier__alpha=alpha)
        full_pipeline.fit(train_X_df,train_y_sr)
        train_errs.append(1-full_pipeline.score(train_X_df,train_y_sr))
        full_pipeline.predict(val_X_df)
        val_err=(1-full_pipeline.score(val_X_df,val_y_sr))*100
        val_errs.append(val_err)
        if( best_val_err > val_err):
            best_val_err= val_err
            best_alpha= alpha
            best_k_bin = bins

In [33]:
full_pipeline.set_params(pipeline__Runtime_Transformer__k_bin=best_k_bin, mlpclassifier__alpha=best_alpha)
(1-full_pipeline.score(val_X_df,val_y_sr))

0.4861727552065551

In [69]:
full_pipeline.fit(train_X_df,train_y_sr)
print(full_pipeline.score(train_X_df,train_y_sr)*100)
full_pipeline.predict(val_X_df)
print(full_pipeline.score(val_X_df,val_y_sr)*100)

56.394498097746556
50.1877773984295


### IV. KHÓ KHĂN VÀ HẠN CHẾ

<ul> Cơ bản là việc tìm chủ đề cho một câu hỏi phù hợp là rất khó, đây là công đoạn mất thời gian nhất. Và việc khi tìm được chủ đề, nghĩ xem dữ liệu đó có thích hợp hay không, dữ liệu đó có sạch và đủ lớn phục vụ bài toán không thì đúng là một vấn đề lớn của nhóm. Hên tìm được một nguồn dữ liệu cũng khá ổn về độ tin tưởng, do cộng động mạng bảo vậy.<br>
Khi đã lấy dữ liệu về, vấn đề hiểu dữ liệu và dữ liệu đó thật sự có quan trọng không cũng là một vấn đề, ví dụ như trong nhóm, dữ liệu phim thì có nhiều cột, nhưng lấy được vài dữ liệu về các thông tin cơ bản đã nói ở trên. Về việc phân tích một bộ phim thì đó quá ít để đánh giá phim hay hay không hay, vì nội dung, kỹ xảo mới nói lên tất cả, tuy nhiên, trong khả năng có hạn thì nhóm không làm được vậy.</ul>

<ul>
    <b><u>Hướng phát triển:</u></b>
        Về cơ bản, phim thường được đánh giá chính bởi nội dung nên việc chúng ta phân tích nội dụng bằng các từ trọng tâm và so sánh với xu hướng của những từ này với thế giới thực thì sẽ có vài ý nghĩa to lớn để đánh giá phim, kiểu như giờ người ta thích mấy phim kiểu phản diện ban đầu rồi lúc sao lật mặt làm chính diện. Thứ hai là thông tin kỹ xảo và giá trị đầu tư của một bộ phim, không phải tất cả, nhưng nếu phim được đầu tư hàng tỉ USD thì chất lương sẽ ổn áp hơn nhiều. Do đó để phát triển hơn về bài toán này, nhóm sẽ tự trang bị kiến thức đầy đủ và kỹ năng để phân tích sâu hơn và cố gắng tìm nhiều nguồn dữ liệu sạch và ý nghĩa hơn.
</ul>

### V. TÀI LIỆU KHAM KHẢO
<ul><b>
    <a href="https://www.dienmayxanh.com/kinh-nghiem-hay/diem-imdb-la-gi-cach-chon-phim-hay-dua-tren-diem-i-1201923">IMBD là gì?</a><br>
    <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html">KBinsDiscretizer trong sklearn</a>



</b>
</ul>